# EDA de **Telefonía móvil: Llamadas Salientes**

Notebook preparado para **Visual Studio/VS Code** con Python, con explicaciones en Markdown entre pasos y guardado del CSV 'limpio'.

**Fuente de datos (ruta Windows):**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Llamadas Salientes_con_ID.csv`


## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (11, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
Se preserva la codificación **UTF‑8 con BOM** para acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Llamadas Salientes_con_ID.csv'

# Cargar el archivo
df = pd.read_csv(ruta, encoding='utf-8-sig')

# Copia del crudo
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial del dataset
- Tipos de datos y memoria (`df.info()`).
- Valores faltantes y duplicados.
- Estadísticos descriptivos básicos.

In [ ]:
df.info()
print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

if 'Total de llamadas salientes' in df.columns:
    print('
Estadísticos de Total de llamadas salientes:')
    print(df['Total de llamadas salientes'].describe())

## 4. Validaciones de consistencia
- Conversión a numérico.
- Chequeo de negativos.
- Verificación **pospago + prepago = total**.


In [ ]:
num_cols = ['Registro_ID','Año','Trimestre','Llamadas pospago salientes','Llamadas prepago salientes','Total de llamadas salientes','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

# Consistencia pospago+prepago vs total
suma = df['Llamadas pospago salientes'] + df['Llamadas prepago salientes']
consist = (suma == df['Total de llamadas salientes'])
print(f'
Registros consistentes: {consist.mean():.2%} ({consist.sum()} de {len(consist)})')
df['dif_llamadas'] = df['Total de llamadas salientes'] - suma

## 5. Limpieza y ordenamiento
- Eliminación de duplicados.
- Orden temporal por `Año` y `Trimestre`.
- Construcción de `Periodo` (YYYY-Qn).
- Remoción de columnas auxiliares.

In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre']).reset_index(drop=True)
df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)
df = df.drop(columns=['dif_llamadas'], errors='ignore')
display(df.head())

## 6. Variables derivadas
- `porc_pospago_llam` y `porc_prepago_llam` (% sobre el total).
- `Total_llamadas_QoQ_%` (variación trimestral).

In [ ]:
df['porc_pospago_llam'] = df['Llamadas pospago salientes'] / df['Total de llamadas salientes'] * 100
df['porc_prepago_llam'] = df['Llamadas prepago salientes'] / df['Total de llamadas salientes'] * 100
df['Total_llamadas_QoQ_%'] = df['Total de llamadas salientes'].pct_change() * 100

display(df[['Año','Trimestre','porc_pospago_llam','porc_prepago_llam','Total_llamadas_QoQ_%']].tail(8))

## 7. Visualizaciones
- Serie temporal de `Total de llamadas salientes`.
- Participación (%) de **pospago vs prepago** (últimos 12 trimestres).

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Serie temporal de llamadas totales
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Total de llamadas salientes', marker='o', color='#E67E22')
plt.xticks(rotation=45, ha='right')
plt.title('Total de llamadas salientes por trimestre')
plt.xlabel('Periodo'); plt.ylabel('Total de llamadas'); plt.tight_layout(); plt.show()

# 7.2 Porcentaje pospago vs prepago (últimos 12 trimestres)
tail = df.tail(12)
plt.figure()
sns.lineplot(data=tail, x='Periodo', y='porc_pospago_llam', marker='o', label='Pospago', color='#1ABC9C')
sns.lineplot(data=tail, x='Periodo', y='porc_prepago_llam', marker='o', label='Prepago', color='#D35400')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation=45, ha='right')
plt.title('Participación (%) en llamadas: pospago vs prepago (últimos 12 trimestres)')
plt.xlabel('Periodo'); plt.ylabel('% del total de llamadas'); plt.legend(); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)